In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.reset_default_graph()

x = tf.Variable(np.array([
    0, 0, 7, 0, 0, 0, 3, 0, 2,
    2, 0, 0, 0, 0, 5, 0, 1, 0,
    0, 0, 0, 8, 0, 1, 4, 0, 0,
    0, 1, 0, 0, 9, 6, 0, 0, 8,
    7, 6, 0, 0, 0, 0, 0, 4, 9,
    0, 0, 0, 0, 0, 0 ,0, 0, 0,
    0, 0, 0, 1, 0, 3, 0, 0, 0,
    8, 0, 1, 0, 6, 0, 0, 0, 0,
    0, 0, 0 ,7, 0, 0, 0 ,6, 3
]).reshape(9,9))

dom_zero = tf.equal(x, 0)
n = tf.reduce_sum(tf.cast(dom_zero, tf.int32))
dom0 = tf.broadcast_to(dom_zero, (9,9,9))
domH = tf.stack([
    tf.tile(
        tf.reshape(
            tf.logical_not(tf.reduce_any(tf.equal(x, v), 1)),
        (9, 1)),
    (1, 9)) for v in range(1,10)
])
domV = tf.stack([
    tf.reshape(
        tf.tile(
            tf.logical_not(tf.reduce_any(tf.equal(x, v), 0)),
        (9,)),
    (9,9)) for v in range(1,10)
])
domB = tf.reshape(tf.transpose(
    tf.reshape(tf.tile(
        tf.equal(
            tf.transpose(tf.nn.max_pool(
                tf.reshape(
                    tf.stack([
                        tf.cast(tf.equal(x, v), tf.int32)
                            for v in range(1,10)
                    ], axis=2),
                (1,9,9,9)),
                (1,3,3,1),
                (1,3,3,1),
                'VALID',
            ), (3,1,2,0)),
        0), # shape: 9,3,3,1
    (1,1,1,9)), (9,3,3,3,3)),
(0,1,3,2,4)), (9,9,9))
dom = tf.cast(tf.logical_and(
    dom0, tf.logical_and(
        domB, tf.logical_and(
            domH, domV
        )
    )
), tf.int32)
ct0 = tf.where(tf.equal(tf.reduce_sum(dom, 0), 1))
g0 = tf.argmax(tf.gather_nd(tf.transpose(dom, (1,2,0)), ct0), axis=1) + 1

ctV = tf.where(tf.equal(tf.reduce_sum(dom, 1), 1))
gV = tf.argmax(tf.gather_nd(tf.transpose(dom, (0,2,1)), ctV), axis=1)
locV = tf.stack([gV, ctV[:, 1]], axis=1)
valV = ctV[:, 0] + 1

ctH = tf.where(tf.equal(tf.reduce_sum(dom, 2), 1))
gH = tf.argmax(tf.gather_nd(dom, ctH), axis=1)
locH = tf.stack([ctH[:, 1], gH], axis=1)
valH = ctH[:, 0] + 1

blks = [
    tf.constant(np.array([ [0,0], [0,1], [0,2],
                           [1,0], [1,1], [1,2],
                           [2,0], [2,1], [2,2] ]) + blk_ptr)
        for blk_ptr in [[0,0], [0,3], [0,6], [3,0], [3,3], [3,6], [6,0], [6,3], [6,6]]
]
blk_cubes = [ # block*(in-block, vals) = 9*(9,9)
    tf.gather_nd(tf.transpose(dom, (1,2,0)), blk) for blk in blks
]
ctB = [ # values per block: block*(#vals, ) = 9*?
    tf.reshape(tf.where(tf.equal(tf.reduce_sum(blk_cube, axis=0), 1)), [-1])
        for blk_cube in blk_cubes
]
gB = [ # in-block indices: block*(in-block, )
    tf.argmax(tf.gather(tf.transpose(blk_cube), ctb), axis=1)
        for ctb, blk_cube in zip(ctB, blk_cubes)
]
locB = tf.concat([ # coordinate transform
    tf.gather(blk, gb) for gb, blk in zip(gB, blks)
], axis=0)
valB = tf.concat(ctB, axis=0) + 1 # update values

idxUpdates = tf.concat([ct0, locV, locH, locB], axis=0)
valUpdates = tf.concat([g0, valV, valH, valB], axis=0)
reasons = tf.concat([
    tf.fill(tf.shape(g0), tf.constant("constraint")),
    tf.fill(tf.shape(valV), tf.constant("v-scan")),
    tf.fill(tf.shape(valH), tf.constant("h-scan"), ),
    tf.fill(tf.shape(valB), tf.constant("blk-scan"))
], axis=0)

update_op = tf.scatter_nd_update(x, idxUpdates, valUpdates)

init = tf.global_variables_initializer()


In [3]:
with tf.Session() as sess:
    sess.run(init)
    for step in range(sess.run(n)):
        idx, val, reason,  _ = sess.run([
            idxUpdates, valUpdates, reasons, update_op])
        if len(val):
            print("Step", step)
            for i, v, r in zip(idx, val, reason):
                print("[{}, {}] ← {} ({})".format(
                    i[0]+1, i[1]+1, v, r.decode('ascii')))
        else:
            break
    print(sess.run(x))
        

Step 0
[1, 5] ← 4 (constraint)
[1, 1] ← 1 (v-scan)
[6, 9] ← 1 (v-scan)
[6, 6] ← 7 (v-scan)
[1, 1] ← 1 (h-scan)
[9, 7] ← 1 (h-scan)
[3, 5] ← 2 (h-scan)
[8, 2] ← 3 (h-scan)
[1, 1] ← 1 (blk-scan)
[3, 5] ← 2 (blk-scan)
[8, 2] ← 3 (blk-scan)
[9, 7] ← 1 (blk-scan)
Step 1
[1, 6] ← 9 (constraint)
[5, 5] ← 1 (v-scan)
[7, 2] ← 7 (v-scan)
[2, 5] ← 7 (v-scan)
[5, 5] ← 1 (h-scan)
[1, 4] ← 6 (h-scan)
[6, 7] ← 6 (h-scan)
[2, 5] ← 7 (blk-scan)
[5, 5] ← 1 (blk-scan)
[6, 7] ← 6 (blk-scan)
[7, 2] ← 7 (blk-scan)
Step 2
[2, 4] ← 3 (constraint)
[2, 9] ← 6 (constraint)
[6, 5] ← 3 (v-scan)
[8, 4] ← 9 (v-scan)
[2, 4] ← 3 (blk-scan)
[8, 4] ← 9 (blk-scan)
Step 3
[4, 8] ← 3 (v-scan)
[5, 3] ← 3 (h-scan)
[5, 6] ← 8 (blk-scan)
[4, 8] ← 3 (blk-scan)
Step 4
[3, 1] ← 3 (v-scan)
[3, 1] ← 3 (h-scan)
[4, 7] ← 7 (h-scan)
[9, 5] ← 8 (h-scan)
[3, 1] ← 3 (blk-scan)
[4, 7] ← 7 (blk-scan)
Step 5
[7, 5] ← 5 (constraint)
[7, 5] ← 5 (v-scan)
[7, 1] ← 6 (v-scan)
[3, 3] ← 6 (h-scan)
[3, 3] ← 6 (blk-scan)
[7, 5] ← 5 (blk-scan)
Step 6